In [1]:
!nvidia-smi

Sat Nov 16 11:53:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   43C    P8              4W /   60W |      64MiB /   6144MiB |     14%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Install YOLOv8

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [2]:
# Pip install method (recommended)
#!pip install ultralytics==8.2.103 -q

import ultralytics
import torch
#ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image

import os
ROOT_DIR = os.getcwd()
print(ROOT_DIR)

/home/ericles/Documents/Aprendizado/Unicamp/Machine-learning/Pupil-seg


## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [3]:
model = YOLO(f'{ROOT_DIR}/yolov8m-seg.pt')

#results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)
#results[0].boxes.xyxy
#results[0].boxes.conf
#results[0].boxes.cls
#results[0].masks.data

## Custom Training

In [4]:
model.train(data=os.path.join(ROOT_DIR,'data.yaml'), epochs=30, batch=8)

#!yolo task=segment mode=train model=yolov8m-seg.pt data={dataset_location}/data.yaml epochs=10 imgsz=640

New https://pypi.org/project/ultralytics/8.3.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.28 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 5834MiB)
engine/trainer: task=segment, mode=train, model=/home/ericles/Documents/Aprendizado/Unicamp/Machine-learning/Pupil-seg/yolov8m-seg.pt, data=/home/ericles/Documents/Aprendizado/Unicamp/Machine-learning/Pupil-seg/data.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None

E0000 00:00:1731770348.688440    9946 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731770348.698962    9946 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

train: Scanning /home/ericles/Documents/Aprendizado/Unicamp/Machine-learning/Pupil-seg/data/labels/train... 5010 image

train: New cache created: /home/ericles/Documents/Aprendizado/Unicamp/Machine-learning/Pupil-seg/data/labels/train.cache


...

                   all        879       1649      0.961      0.824      0.914      0.449      0.862      0.742      0.778      0.343



30 epochs completed in 2.402 hours.
Optimizer stripped from runs/segment/train10/weights/last.pt, 54.8MB
Optimizer stripped from runs/segment/train10/weights/best.pt, 54.8MB

Validating runs/segment/train10/weights/best.pt...
Ultralytics 8.3.28 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 5834MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,222,963 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mA


                   all        879       1649      0.966      0.945      0.974        0.5      0.902      0.884      0.872      0.374
Speed: 0.2ms preprocess, 12.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/segment/train10


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a5c4e0201a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [5]:
model.save('trained.pt')

In [ ]:
model = YOLO('trained.pt')

In [ ]:
test_result = model('data/images/train/frame_000399.PNG')

In [ ]:
x = test_result[0].masks.xy[0]d
y = test_result[0].masks.xy[0][1,:]
len(test_result[0].masks.xy[0][:,0])

## Live webcam

In [ ]:
import cv2
import time

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while cap.isOpened():
    suc, frame = cap.read()

    if suc:
        start = time.perf_counter()
        results = model(frame)
        end = time.perf_counter()

        annotated_frame = results[0].plot()

        cv2.imshow("yolov8 inference", annotated_frame)
        cv2.waitKey()